In [1]:
import pandas as pd
import numpy as np
from calculadora_targets import CalculadoraTargets

In [2]:
def gerar_dados(
    n_contratos: int = 10000, 
    n_meses: int = 24, 
    data_inicio: str = '2022-01-01'
) -> pd.DataFrame:
    """
    Gera um DataFrame robusto simulando a evolução de uma carteira de crédito.

    Cria `n_contratos` únicos e acompanha sua evolução de atraso ao longo de `n_meses`.

    Parâmetros
    ----------
    n_contratos : int
        Número de contratos únicos na carteira.
    n_meses : int
        Número de meses para simular a evolução.
    data_inicio : str
        Data de início da simulação (formato 'YYYY-MM-DD').

    Retorna
    -------
    pd.DataFrame
        DataFrame com as colunas ['ccb', 'anomes', 'atraso'].
    """
    print(f"Gerando dados para {n_contratos} contratos ao longo de {n_meses} meses...")

    # Cria a lista de meses no formato YYYYMM
    meses = pd.to_datetime(
        pd.date_range(start=data_inicio, periods=n_meses, freq='MS')
    ).strftime('%Y%m')

    # Cria a estrutura base do DataFrame com todos os contratos para todos os meses
    contratos_ids = range(1, n_contratos + 1)
    df = pd.DataFrame(
        pd.MultiIndex.from_product([contratos_ids, meses], names=['ccb', 'anomes']).to_frame(index=False)
    )
    # Garante a ordenação correta, essencial para a lógica de evolução
    df = df.sort_values(['ccb', 'anomes']).reset_index(drop=True)
    df['atraso'] = 0 # Inicializa atraso com 0

    # --- Mês 1: Define o status inicial dos contratos ---
    prob_inicio = [0.92, 0.05, 0.02, 0.01]  # 92% começam em dia
    atraso_inicio = [0, 15, 30, 90]
    primeiro_mes_mask = df['anomes'] == meses[0]
    df.loc[primeiro_mes_mask, 'atraso'] = np.random.choice(
        atraso_inicio, size=n_contratos, p=prob_inicio
    )

    # --- Meses 2 a N: Simula a evolução mês a mês ---
    for i in range(1, len(meses)):
        anomes_atual = meses[i]
        anomes_anterior = meses[i-1]

        # Pega os valores de atraso do mês anterior de forma eficiente
        atraso_anterior = df.loc[df['anomes'] == anomes_anterior, 'atraso'].values
        atraso_atual = atraso_anterior.copy()

        # 1. Agravamento: 85% de chance de o atraso aumentar para quem já estava atrasado
        mask_agrava = (atraso_anterior > 0) & (np.random.rand(n_contratos) < 0.85)
        aumento = np.random.randint(25, 35, size=mask_agrava.sum())
        atraso_atual[mask_agrava] += aumento

        # 2. Cura: 5% de chance de um contrato atrasado ser regularizado
        mask_cura = (atraso_anterior > 0) & (np.random.rand(n_contratos) < 0.05)
        atraso_atual[mask_cura] = 0

        # 3. Nova Inadimplência: 2% de chance de um contrato em dia entrar em atraso
        mask_nova_inadimplencia = (atraso_anterior == 0) & (np.random.rand(n_contratos) < 0.02)
        atraso_atual[mask_nova_inadimplencia] = np.random.randint(1, 15, size=mask_nova_inadimplencia.sum())
        
        # Atribui os novos valores de atraso ao DataFrame
        df.loc[df['anomes'] == anomes_atual, 'atraso'] = atraso_atual

    df['ccb']=df['ccb'].astype(str).str.zfill(8)
    print("Dados gerados com sucesso!")
    return df.astype({'atraso': 'int32'})

In [3]:
# PASSO 1: Gerar o conjunto de dados robusto
# (10.000 contratos * 24 meses = 240.000 registros)
dados = gerar_dados(n_contratos=10000, n_meses=24)

# Mostra informações sobre o DataFrame gerado
print("\n--- Informações do DataFrame Gerado ---")
dados.info()
print("\n--- Primeiros Registros ---")
display(dados.head())
print("\n--- Últimos Registros ---")
display(dados.tail())

Gerando dados para 10000 contratos ao longo de 24 meses...
Dados gerados com sucesso!

--- Informações do DataFrame Gerado ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ccb     240000 non-null  object
 1   anomes  240000 non-null  object
 2   atraso  240000 non-null  int32 
dtypes: int32(1), object(2)
memory usage: 4.6+ MB

--- Primeiros Registros ---


,ccb,anomes,atraso
0,00000001,202201,0
1,00000001,202202,0
2,00000001,202203,0
3,00000001,202204,0
4,00000001,202205,0



--- Últimos Registros ---


,ccb,anomes,atraso
239995,00010000,202308,445
239996,00010000,202309,475
239997,00010000,202310,475
239998,00010000,202311,504
239999,00010000,202312,0


In [ ]:
# 2. Defina as metas que você quer calcular
#    Formato: (tipo, dias_de_atraso, horizonte_em_meses)
minhas_metas = [
    ('ever', 90, 12),   # Atingiu 90 dias de atraso em algum momento nos próximos 12 meses?
    ('over', 90, 12),    # Estava com 90 dias ou mais de atraso exatamente 12 meses depois?
    ('ever', 30, 4),    # Atingiu 30 dias de atraso em algum momento nos próximos 4 meses?
    ('over', 30, 4),    # Estava com 30 dias ou mais de atraso exatamente 4 meses depois?
]

# 3. Crie a calculadora e execute o cálculo
#    Instancie a classe com suas especificações.
calculadora = CalculadoraTargets(
    specs=minhas_metas, 
    date_col='anomes',
    contract_col='ccb',
    dpd_col='atraso'
    )

<>:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
C:\Users\JM\AppData\Local\Temp\ipykernel_3212\2915004286.py:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('ever', 30, 4)    # Atingiu 30 dias de atraso em algum momento nos próximos 4 meses?
C:\Users\JM\AppData\Local\Temp\ipykernel_3212\2915004286.py:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('ever', 30, 4)    # Atingiu 30 dias de atraso em algum momento nos próximos 4 meses?
C:\Users\JM\AppData\Local\Temp\ipykernel_3212\2915004286.py:6: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('ever', 30, 4)    # Atingiu 30 dias de atraso em algum momento nos próximos 4 meses?


TypeError: 'tuple' object is not callable

In [ ]:
#    Chame o método 'calcular' para obter o resultado.
df = calculadora.calcular(dados)

# 4. Veja o resultado!
display(df)

,ccb,anomes,atraso,ever90m12,over90m12
0,00000001,202201,0,0,0
1,00000001,202202,0,0,0
2,00000001,202203,0,0,0
3,00000001,202204,0,0,0
4,00000001,202205,0,0,0
...,...,...,...,...,...
239995,00010000,202308,0,0,0
239996,00010000,202309,0,0,0
239997,00010000,202310,0,0,0
239998,00010000,202311,0,0,0


In [ ]:
df['ever90m12'].sum()

58055

In [ ]:
fig = calculadora.plot_targets(
    df,
    target_cols=[
        "ever90m12",
        "over90m12",
        "ever30m4",
        "over30m4",
        ],
    normalize=True,   # exibe porcentagem
    title="Evolução dos targets"
)
fig.show()
